In [1]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

In [2]:
# data_df['IC50_nM'].describe().T.head()

In [3]:
#data_df = pd.read_csv("low_503.csv")
#data_df = pd.read_csv('pca_Q1_173.csv')
data_df = pd.read_csv('ori_729.csv')

In [4]:
data_df = pd.read_csv('ori_729.csv')
data_df.drop(labels='pIC50',inplace=True,axis=1)##删掉这玩意
data_df_y = data_df['IC50_nM']
data_df_x = data_df.drop(labels=['SMILES','IC50_nM'],axis=1)
data_df_x.shape
X,y = data_df_x , data_df_y
xlist = X.columns

In [11]:
data_df_x.shape
data_df['nB'].unique()

array([0], dtype=int64)

## 1.MIC最大互信息

In [6]:
#mic最大信息
from minepy import MINE
import operator
#算相关性的矩阵
coff_data_df = data_df.drop(labels='SMILES',axis=1)
features = data_df_x.columns #只取 x

mine = MINE()
mic_scores = []
for feature_name in features:
    mine.compute_score(coff_data_df.loc[:,feature_name],data_df_y)
    mic_scores.append(mine.mic())

In [7]:
mic_result = sorted(list(zip(features,mic_scores)),key=operator.itemgetter(1),reverse=True)
mic_sorted_attrs = [x[0] for x in mic_result]
mic_ranking = [mic_sorted_attrs.index(x)+1 for x in features]
print("# MIC result")
print(mic_sorted_attrs[0:40])
#print(mic_ranking)
print('---------------------------------------------')
print(mic_result[0:40])

# MIC result
['BCUTc-1l', 'BCUTc-1h', 'MDEC-23', 'minHsOH', 'SsOH', 'WTPT-5', 'SHsOH', 'LipoaffinityIndex', 'maxHsOH', 'MLogP', 'maxsOH', 'minsOH', 'MDEC-22', 'nC', 'maxHBa', 'gmax', 'TopoPSA', 'VABC', 'McGowan_Volume', 'MLFER_A', 'Kier1', 'apol', 'ATSp5', 'AMR', 'ETA_Eta_R', 'VP-0', 'BCUTp-1l', 'SP-0', 'BCUTw-1h', 'WTPT-1', 'ETA_Alpha', 'CrippenMR', 'C1SP2', 'SaaCH', 'SP-1', 'ETA_Eta_R_L', 'fragC', 'nHeavyAtom', 'VAdjMat', 'Zagreb']
---------------------------------------------
[('BCUTc-1l', 0.3938375609903433), ('BCUTc-1h', 0.3717326124796673), ('MDEC-23', 0.3393809665239051), ('minHsOH', 0.32999677193878485), ('SsOH', 0.3299709480067496), ('WTPT-5', 0.32962669655357907), ('SHsOH', 0.32567884478317394), ('LipoaffinityIndex', 0.319822104324189), ('maxHsOH', 0.3194005587580585), ('MLogP', 0.31936945065239764), ('maxsOH', 0.31571793052943853), ('minsOH', 0.30990708269191874), ('MDEC-22', 0.3086164449850388), ('nC', 0.30801101773263134), ('maxHBa', 0.3054198597730482), ('gmax', 0.3051222

## 2.皮尔森系数

In [8]:
from scipy.stats import pearsonr
y_values = data_df_y.values
pearsonr_feature_importance = [(column,pearsonr(data_df_x[column],y_values)[0]) for column in features ]
pearsonr_feature_importance.sort(key=lambda x:abs(x[1]), reverse=True)
pearson_sorted_attrs = [item[0] for item in pearsonr_feature_importance]
pearson_ranking = [ pearson_sorted_attrs . index(x) + 1 for x in features]   
print("# Pearson result")
print(pearson_sorted_attrs[0:40])
print('---------------------------------------------')
print(pearsonr_feature_importance[0:40])

# Pearson result
['nB', 'ATSc4', 'ATSc5', 'naAromAtom', 'nAromBond', 'CrippenLogP', 'nwHBa', 'SP-5', 'AMR', 'SP-7', 'nC', 'SP-4', 'CrippenMR', 'SP-1', 'ATSm2', 'ALogP', 'SP-3', 'ATSp1', 'ECCEN', 'ATSp2', 'nHBint6', 'nHBint10', 'nO', 'VP-1', 'SP-6', 'VP-4', 'nHBint4', 'nHeavyAtom', 'VP-5', 'ATSp3', 'VP-3', 'nX', 'ATSm3', 'apol', 'ATSp4', 'VP-0', 'nF', 'SP-2', 'SP-0', 'ATSm4']
---------------------------------------------
[('nB', nan), ('ATSc4', -0.0744746970407737), ('ATSc5', 0.07376303272484971), ('naAromAtom', -0.0665569666181238), ('nAromBond', -0.06549752562789408), ('CrippenLogP', -0.06240116330408948), ('nwHBa', -0.06085578444499766), ('SP-5', -0.03970990391194856), ('AMR', -0.039037846035027586), ('SP-7', -0.03856232686469302), ('nC', -0.03695038353417611), ('SP-4', -0.03452670529117235), ('CrippenMR', -0.03385996134396934), ('SP-1', -0.03383842882765993), ('ATSm2', -0.033779694080272954), ('ALogP', -0.03359041994709491), ('SP-3', -0.03304196444020681), ('ATSp1', -0.0326016179352

C:\Users\wtf\Anaconda3\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


## 3.两者取交集

In [9]:
pcc_set = set(pearson_sorted_attrs[0:80])
mic_set = set(mic_sorted_attrs[0:80])
pcc_set&mic_set

{'AMR',
 'ATSm2',
 'ATSp1',
 'ATSp2',
 'ATSp4',
 'ATSp5',
 'BCUTw-1h',
 'CrippenMR',
 'ECCEN',
 'SP-0',
 'SP-1',
 'SP-2',
 'SP-3',
 'SP-4',
 'SP-5',
 'SP-6',
 'SP-7',
 'VP-0',
 'VP-1',
 'VP-2',
 'apol',
 'nAtom',
 'nBase',
 'nC',
 'nHeavyAtom'}

In [10]:
### --------------广雨部分在上----------------------------------------
### -----------------------------------------------------------------
--------------------------------------------------------------------

SyntaxError: invalid syntax (<ipython-input-10-3bb8e2c6a13f>, line 3)

In [ ]:
from sklearn.metrics import r2_score,mean_squared_error

def evaluate(data,model):
    pre = model.predict(data)
    pre_res = [round(res) for res in pre]
    y = data.get_label()
    acc = r2_score(y,pre_res)
    return acc
#先定义评价函数
def print_r2_score(model,x_train,y_train,x_test,y_test):
    print(model.score(x_train,y_train))
    print(model.score(x_test,y_test))

In [ ]:
### 标准化

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
#标准化 0.79
#非标准化 0.80

# std1 = MinMaxScaler()
# X=std1.fit_transform(X)
# std2 = MinMaxScaler()
# y=std2.fit_transform(y.values.reshape(-1,1))
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=666)

# std1 = StandardScaler()
# x_train = std1.fit_transform(x_train)
# x_test = std1.transform(x_test)
# std2 = StandardScaler()
# y_train = std2.fit_transform(y_train.values.reshape(-1,1))  # 必须传二维
# y_test = std2.transform(y_test.values.reshape(-1,1))

In [ ]:
%%time
import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBRegressor as XGBR
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold, cross_val_score as CVS
from sklearn.metrics import mean_squared_error,r2_score
 

#3. 网格搜索法


#1.sklearn实现法
xgbr = XGBR(n_estimators=100,max_depth=8, n_jobs=-1,learning_rate=0.15)
xgbr.fit(x_train, y_train,
        eval_set=[(x_train, y_train), (x_test, y_test)],
        eval_metric='rmse',
        verbose=False)
evals_result = xgbr.evals_result()#fit中有eval_metric

print_r2_score(xgbr,x_train,y_train,x_test,y_test)
#CVS(xgbr,x_train, y_train,cv=10).mean()

#2.xgb实现法
# dtrain = xgb.DMatrix(x_train, y_train)
# dtest = xgb.DMatrix(x_test, y_test)
# param = {'verbosity':0
#          , 'n_jobs' :-1
#          ,'max_depth':3 
#          ,'subsample':1,
#          'min_child_weight':1
#          ,'eta':0.1
#          ,'objective': 'reg:linear'
#          ,'nfold':5
#          }

# param1 = {'verbosity':0
#          , 'n_jobs' :-1
#          ,'max_depth':2 
#          ,'subsample':1
#          ,"lambda" :1
#          ,'eta':0.3
#          ,'objective': 'reg:linear'
#          ,'nfold':5
#          }

# num_round = 1000
# bst=xgb.cv(param , dtrain ,num_round)
# #bst1=xgb.cv(param1 , dtrain ,num_round)

In [ ]:
# import sklearn
# sorted(sklearn.metrics.SCORERS.keys())

## 3.网格搜索法

In [ ]:
# %%time
# # 1.选出 n_estimators
# from sklearn.model_selection import GridSearchCV
# xgbr = XGBR()#1.定义分类器
# tuned_parameters = {'n_estimators':[100,200]
#                     ,'max_depth':[3]
#                     ,'learning_rate':[0.05,0.1,0.15,0.2]}
# #2.设置可调参数
# reg = GridSearchCV(xgbr,tuned_parameters
#                    ,scoring='neg_mean_squared_error',cv=5)
# reg.fit(x_train,y_train)
# reg.cv_results_
# print(-reg.best_score_,reg.best_params_)

In [ ]:
#xgbr.feature_importances_

In [ ]:
im=pd.DataFrame({'importance':xgbr.feature_importances_,'var':xlist})
im=im.sort_values(by='importance',ascending=False)
im.head(20)

In [ ]:
xgb.plot_importance(xgbr,max_num_features=20,importance_type='gain')

In [ ]:
xgb_sel=set(label[0:80])
xgb_sel&mic_set

In [ ]:
# ytrain_pred=xgbr.predict(Xtrain)
# ytest_pred = xgbr.predict(Xtest)
# r2_train = r2_score(ytrain_pred,ytrain)
# r2_test = r2_score(ytest_pred,ytest)
# print(r2_train)
# print(r2_test)